In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 50)
from scipy.spatial import cKDTree
from math import *
import matplotlib.pyplot as plt
%matplotlib inline
np.random.seed(42)
import urllib.request
import urllib, os

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd
import xarray as xr
import math
from datetime import date, timedelta
from dateutil.relativedelta import relativedelta

In [4]:
# Importing libraries
import os, shutil
from keras import models, layers, optimizers, regularizers
import datetime
import matplotlib.pyplot as plt
import scipy
import numpy as np
from PIL import Image
from scipy import ndimage
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import itertools
from sklearn.metrics import confusion_matrix, classification_report
np.random.seed(123)
import math
import tensorflow as tf
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.optimizers import RMSprop
#from keras.applications import VGG19
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

In [5]:
df_grace = xr.open_dataset(r'/content/drive/MyDrive/ugp/2_deg/regrid_2_deg/grace_04-2002_11-2021.nc')
df_preci = xr.open_dataset(r'/content/drive/MyDrive/ugp/2_deg/regrid_2_deg/Precipitation_04-2002_11-2021.nc' )
df_temp = xr.open_dataset(r'/content/drive/MyDrive/ugp/2_deg/regrid_2_deg/Land_and_sea_temp_04-2002_11-2021.nc')
df_ssh = xr.open_dataset(r'/content/drive/MyDrive/ugp/2_deg/regrid_2_deg/zos_AVISO_04-2002_12-2010.nc' )

In [6]:
df_preci = xr.open_dataset(r'/content/drive/MyDrive/ugp/2_deg/regrid_2_deg/Precipitation_04-2002_11-2021.nc' )
df_preci

<xarray.Dataset>
Dimensions:        (lat: 90, lon: 180, time: 236)
Coordinates:
  * time           (time) datetime64[ns] 2002-04-01 2002-05-01 ... 2021-11-01
  * lon            (lon) int64 0 2 4 6 8 10 12 ... 346 348 350 352 354 356 358
  * lat            (lat) float64 -90.0 -88.0 -86.0 -84.0 ... 82.0 84.0 86.0 88.0
Data variables:
    precipitation  (time, lat, lon) float64 ...

In [7]:
df_grace

<xarray.Dataset>
Dimensions:        (lat: 90, lon: 180, time: 236)
Coordinates:
  * time           (time) datetime64[ns] 2002-04-01 2002-05-01 ... 2021-11-01
  * lon            (lon) int64 0 2 4 6 8 10 12 ... 346 348 350 352 354 356 358
  * lat            (lat) float64 -90.0 -88.0 -86.0 -84.0 ... 82.0 84.0 86.0 88.0
Data variables:
    lwe_thickness  (time, lat, lon) float64 ...

In [8]:
df_grace = df_grace['lwe_thickness'][:,25:66,10:71]
df_grace

<xarray.DataArray 'lwe_thickness' (time: 236, lat: 41, lon: 61)>
[590236 values with dtype=float64]
Coordinates:
  * time     (time) datetime64[ns] 2002-04-01 2002-05-01 ... 2021-11-01
  * lon      (lon) int64 20 22 24 26 28 30 32 34 ... 128 130 132 134 136 138 140
  * lat      (lat) float64 -40.0 -38.0 -36.0 -34.0 -32.0 ... 34.0 36.0 38.0 40.0

In [9]:
df_grace[0]

<xarray.DataArray 'lwe_thickness' (lat: 41, lon: 61)>
array([[ -4.820502,  -4.86268 ,  -4.722223, ...,  -1.114338,  -1.147498,
         -1.73444 ],
       [ -3.360285,  -4.353694,  -4.195386, ...,  -1.221529,  -1.99629 ,
         -3.216284],
       [ -1.494723,  -2.233424,  -3.287187, ...,  -3.25616 ,  -3.630314,
         -1.493465],
       ...,
       [  1.336078,   2.07022 ,   3.497176, ...,   1.111395,   0.682665,
          1.516902],
       [  1.413195,   8.259285,   8.574983, ...,  -3.197883,  -0.446197,
         27.000797],
       [  7.10087 ,  10.261107,  10.605567, ...,  -1.413481, -18.366465,
         -7.015009]])
Coordinates:
    time     datetime64[ns] 2002-04-01
  * lon      (lon) int64 20 22 24 26 28 30 32 34 ... 128 130 132 134 136 138 140
  * lat      (lat) float64 -40.0 -38.0 -36.0 -34.0 -32.0 ... 34.0 36.0 38.0 40.0

In [10]:
df_preci = df_preci['precipitation'][:,25:66,10:71]
df_preci

<xarray.DataArray 'precipitation' (time: 236, lat: 41, lon: 61)>
[590236 values with dtype=float64]
Coordinates:
  * time     (time) datetime64[ns] 2002-04-01 2002-05-01 ... 2021-11-01
  * lon      (lon) int64 20 22 24 26 28 30 32 34 ... 128 130 132 134 136 138 140
  * lat      (lat) float64 -40.0 -38.0 -36.0 -34.0 -32.0 ... 34.0 36.0 38.0 40.0

In [11]:
df_preci[0]

<xarray.DataArray 'precipitation' (lat: 41, lon: 61)>
array([[2.446428, 2.346867, 1.993499, ..., 0.684222, 0.578498, 0.589409],
       [1.524702, 1.621783, 1.537544, ..., 0.528166, 0.456726, 0.445193],
       [0.85676 , 0.958198, 1.021826, ..., 0.463121, 0.411438, 0.35833 ],
       ...,
       [1.972414, 2.29273 , 1.837716, ..., 4.396911, 3.6905  , 3.253795],
       [1.885994, 2.106746, 2.204469, ..., 3.292836, 3.594714, 3.648915],
       [1.695725, 2.141913, 2.767659, ..., 2.042926, 2.716503, 3.196219]])
Coordinates:
    time     datetime64[ns] 2002-04-01
  * lon      (lon) int64 20 22 24 26 28 30 32 34 ... 128 130 132 134 136 138 140
  * lat      (lat) float64 -40.0 -38.0 -36.0 -34.0 -32.0 ... 34.0 36.0 38.0 40.0

In [12]:
df_temp = df_temp['temperature'][:,25:66,10:71]
df_temp

<xarray.DataArray 'temperature' (time: 236, lat: 41, lon: 61)>
[590236 values with dtype=float64]
Coordinates:
  * time     (time) datetime64[ns] 2002-04-01 2002-05-01 ... 2021-11-01
  * lon      (lon) int64 20 22 24 26 28 30 32 34 ... 128 130 132 134 136 138 140
  * lat      (lat) float64 -40.0 -38.0 -36.0 -34.0 -32.0 ... 34.0 36.0 38.0 40.0

In [13]:
df_temp[0]

<xarray.DataArray 'temperature' (lat: 41, lon: 61)>
array([[ 1.352317,  1.144489,  1.058138, ..., -0.335429, -0.251992, -0.139329],
       [ 1.027193,  0.755376,  0.648633, ..., -0.48993 , -0.417516, -0.208664],
       [ 1.065096,  0.791193,  0.677843, ..., -0.493843, -0.416519, -0.260861],
       ...,
       [ 0.778801,  0.843056,  0.918395, ...,  1.937746,  2.728627,  2.385574],
       [ 0.985081,  1.394915,  1.164222, ...,  1.145482,  1.264355,  2.510565],
       [ 1.887165,  2.262666,  1.371051, ...,  1.0092  ,  0.794397,  1.607426]])
Coordinates:
    time     datetime64[ns] 2002-04-01
  * lon      (lon) int64 20 22 24 26 28 30 32 34 ... 128 130 132 134 136 138 140
  * lat      (lat) float64 -40.0 -38.0 -36.0 -34.0 -32.0 ... 34.0 36.0 38.0 40.0

In [14]:
df_ssh = df_ssh['zos'][:,25:66,10:71]
df_ssh

<xarray.DataArray 'zos' (time: 105, lat: 41, lon: 61)>
[262605 values with dtype=float64]
Coordinates:
  * time     (time) datetime64[ns] 2002-04-01 2002-05-01 ... 2010-12-01
  * lon      (lon) int64 20 22 24 26 28 30 32 34 ... 128 130 132 134 136 138 140
  * lat      (lat) float64 -40.0 -38.0 -36.0 -34.0 -32.0 ... 34.0 36.0 38.0 40.0

In [15]:
df_ssh[0]

<xarray.DataArray 'zos' (lat: 41, lon: 61)>
array([[0.863975, 0.732271, 0.409582, ..., 0.494863, 0.483598, 0.476015],
       [0.756291, 1.010434, 0.971179, ..., 0.506403, 0.491956, 0.489491],
       [0.297212, 0.558802, 0.917073, ..., 0.508088, 0.495943,      nan],
       ...,
       [     nan,      nan,      nan, ..., 0.653466,      nan,      nan],
       [     nan,      nan,      nan, ..., 0.536126, 0.537222,      nan],
       [     nan,      nan,      nan, ..., 0.440976, 0.448936, 0.438511]])
Coordinates:
    time     datetime64[ns] 2002-04-01
  * lon      (lon) int64 20 22 24 26 28 30 32 34 ... 128 130 132 134 136 138 140
  * lat      (lat) float64 -40.0 -38.0 -36.0 -34.0 -32.0 ... 34.0 36.0 38.0 40.0

##Data Interpolation

####SSH

In [16]:
import numpy as np
import scipy.interpolate


In [17]:
loni = np.array(df_ssh['lon'])
lati = np.array(df_ssh['lat'])

In [18]:
X, Y = np.meshgrid(loni, lati)
XI, YI = np.meshgrid(loni,lati)

In [19]:
for i in range(0,105):
  df_ssh[i] = scipy.interpolate.griddata((X.flatten(),Y.flatten()),np.array(df_ssh[i]).flatten() , (XI,YI),method='cubic')

###Grace

In [20]:
df_grace

<xarray.DataArray 'lwe_thickness' (time: 236, lat: 41, lon: 61)>
[590236 values with dtype=float64]
Coordinates:
  * time     (time) datetime64[ns] 2002-04-01 2002-05-01 ... 2021-11-01
  * lon      (lon) int64 20 22 24 26 28 30 32 34 ... 128 130 132 134 136 138 140
  * lat      (lat) float64 -40.0 -38.0 -36.0 -34.0 -32.0 ... 34.0 36.0 38.0 40.0

In [21]:
loni = np.array(df_grace['lon'])
lati = np.array(df_grace['lat'])

In [22]:
X, Y = np.meshgrid(loni, lati)
XI, YI = np.meshgrid(loni,lati)

In [23]:
for i in range(0,236):
  df_grace[i] = scipy.interpolate.griddata((X.flatten(),Y.flatten()),np.array(df_grace[i]).flatten() , (XI,YI),method='cubic')

###Precip

In [24]:
loni = np.array(df_preci['lon'])
lati = np.array(df_preci['lat'])

In [25]:
X, Y = np.meshgrid(loni, lati)
XI, YI = np.meshgrid(loni,lati)
for i in range(0,236):
  df_preci[i] = scipy.interpolate.griddata((X.flatten(),Y.flatten()),np.array(df_preci[i]).flatten() , (XI,YI),method='cubic')

###Normalization

##SSH

In [26]:
ssh_mini = 999999
ssh_maxi = -999999

In [27]:
for i in range(0,105):
  ssh_maxi = np.maximum(ssh_maxi,np.amax(np.array(df_ssh[i]),where=~np.isnan(np.array(df_ssh[i])), initial=-1))
  ssh_mini = np.minimum(ssh_mini,np.amin(np.array(df_ssh[i]),where=~np.isnan(np.array(df_ssh[i])), initial=-1))

In [28]:
print(ssh_mini, ssh_maxi)

-1.0 -1.0


In [29]:
for i in range(0,105):
  x = df_ssh[i]
  x = (x-ssh_mini)/(ssh_maxi-ssh_mini)
  df_ssh[i] = x

In [30]:
df_ssh[0]

<xarray.DataArray 'zos' (lat: 41, lon: 61)>
array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])
Coordinates:
    time     datetime64[ns] 2002-04-01
  * lon      (lon) int64 20 22 24 26 28 30 32 34 ... 128 130 132 134 136 138 140
  * lat      (lat) float64 -40.0 -38.0 -36.0 -34.0 -32.0 ... 34.0 36.0 38.0 40.0

##Temperature

In [31]:
temp_mini = 999999
temp_maxi = -999999
for i in range(0,236):
  temp_maxi = np.maximum(temp_maxi,np.amax(np.array(df_temp[i]),where=~np.isnan(np.array(df_temp[i])), initial=-1))
  temp_mini = np.minimum(temp_mini,np.amin(np.array(df_temp[i]),where=~np.isnan(np.array(df_temp[i])), initial=-1))
for i in range(0,105):
  x = df_temp[i]
  x = (x-temp_mini)/(temp_maxi-temp_mini)
  df_temp[i] = x

In [32]:
temp_maxi

7.646506613402127

In [33]:
temp_mini

-9.476898891484783

##Precipitation

In [34]:
preci_mini = 999999
preci_maxi = -999999
for i in range(0,236):
  preci_maxi = np.maximum(preci_maxi,np.amax(np.array(df_preci[i]),where=~np.isnan(np.array(df_preci[i])), initial=-1))
  preci_mini = np.minimum(preci_mini,np.amin(np.array(df_preci[i]),where=~np.isnan(np.array(df_preci[i])), initial=-1))
for i in range(0,105):
  x = df_preci[i]
  x = (x-preci_mini)/(preci_maxi-preci_mini)
  df_preci[i] = x

###Grace

In [35]:
grace_mini = 999999
grace_maxi = -999999
for i in range(0,236):
  grace_maxi = np.maximum(grace_maxi,np.amax(np.array(df_grace[i]),where=~np.isnan(np.array(df_grace[i])), initial=-1))
  grace_mini = np.minimum(grace_mini,np.amin(np.array(df_grace[i]),where=~np.isnan(np.array(df_grace[i])), initial=-1))
for i in range(0,105):
  x = df_grace[i]
  x = (x-grace_mini)/(grace_maxi-grace_mini)
  df_grace[i] = x

In [36]:
df_grace= df_grace.fillna(0)
df_preci= df_preci.fillna(0)
df_temp= df_temp.fillna(0)
df_ssh= df_ssh.fillna(0)

##Data procressing

In [37]:
data_0 = np.array([df_grace[0],df_ssh[0], df_temp[0]]).T

In [38]:
data_0.shape

(61, 41, 3)

In [39]:
lat = df_grace['lat']
lon = df_grace['lon']
time = df_grace['time'][0]

In [40]:
m_data = [data_0]
for i in range(1,105):
  data_i = np.array([df_grace[i],df_ssh[i], df_temp[i]]).T
  m_data = np.append(np.array(m_data),[data_i],axis=0)


In [41]:
m_data.shape

(105, 61, 41, 3)

In [42]:
m_data_x = m_data
m_data_y = df_preci[0:105]

In [43]:
X_train, X_test, y_train, y_test = train_test_split(m_data_x, m_data_y, test_size=0.10, random_state=42)

In [44]:
X_train[0].shape

(61, 41, 3)

In [45]:
y_train[0].shape

(41, 61)

In [46]:
X_train, X_val, y_train, y_val = train_test_split(X_train[:], y_train[:], test_size=0.10, random_state=42)

In [47]:
np.random.seed(42)

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='sigmoid',
                        input_shape=(61,41,3)))

model.add(layers.Conv2D(56, (2, 2), activation='sigmoid'))

model.add(layers.Conv2D(64, (2, 2), activation='sigmoid'))


model.add(layers.Flatten())
model.add(layers.Dense(2501, activation='sigmoid'))
model.add(tf.keras.layers.Reshape((41, 61)))

model.compile(loss='mean_squared_error', optimizer="adam", metrics=['mse'])

In [48]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 59, 39, 32)        896       
                                                                 
 conv2d_1 (Conv2D)           (None, 58, 38, 56)        7224      
                                                                 
 conv2d_2 (Conv2D)           (None, 57, 37, 64)        14400     
                                                                 
 flatten (Flatten)           (None, 134976)            0         
                                                                 
 dense (Dense)               (None, 2501)              337577477 
                                                                 
 reshape (Reshape)           (None, 41, 61)            0         
                                                                 
Total params: 337,599,997
Trainable params: 337,599,997


In [49]:
X_train[0].shape

(61, 41, 3)

In [51]:
history = model.fit(X_train,
                    y_train,
                    epochs=10,
                    validation_data=(X_val, y_val))

Epoch 1/10
3/3 [==============================] - 10s 3s/step - loss: 0.0156 - mse: 0.0156 - val_loss: 0.0143 - val_mse: 0.0143
Epoch 2/10
3/3 [==============================] - 10s 3s/step - loss: 0.0156 - mse: 0.0156 - val_loss: 0.0143 - val_mse: 0.0143
Epoch 3/10
3/3 [==============================] - 10s 3s/step - loss: 0.0156 - mse: 0.0156 - val_loss: 0.0143 - val_mse: 0.0143
Epoch 4/10
3/3 [==============================] - 10s 3s/step - loss: 0.0156 - mse: 0.0156 - val_loss: 0.0143 - val_mse: 0.0143
Epoch 5/10
3/3 [==============================] - 10s 3s/step - loss: 0.0156 - mse: 0.0156 - val_loss: 0.0143 - val_mse: 0.0143
Epoch 6/10
3/3 [==============================] - 10s 3s/step - loss: 0.0156 - mse: 0.0156 - val_loss: 0.0143 - val_mse: 0.0143
Epoch 7/10
3/3 [==============================] - 10s 3s/step - loss: 0.0156 - mse: 0.0156 - val_loss: 0.0143 - val_mse: 0.0143
Epoch 8/10
3/3 [==============================] - 10s 3s/step - loss: 0.0156 - mse: 0.0156 - val_loss: 0

In [52]:
model.save_weights('./checkpoints/my_checkpoint')

In [55]:
loss, acc = model.evaluate(X_test, y_test, verbose=2)
print("Restored model, mse: {:5.2f}".format(acc))

1/1 - 0s - loss: 0.0161 - mse: 0.0161 - 493ms/epoch - 493ms/step
Restored model, mse:  0.02
